## Exploratory Analysis



In [13]:
%load_ext autoreload
%autoreload 2
import pandas as pd

df = pd.read_table("https://raw.githubusercontent.com/Cyvhee/SemEval2018-Task3/master/datasets/train/SemEval2018-T3-train-taskA.txt", index_col=0, names=["index", "label", "tweet"], skiprows=1)


df_test = pd.read_table("https://raw.githubusercontent.com/Cyvhee/SemEval2018-Task3/master/datasets/goldtest_TaskA/SemEval2018-T3_gold_test_taskA_emoji.txt", index_col=0, names=["index", "label", "tweet"], skiprows=1)

df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,label,tweet
index,,
1,1,Sweet United Nations video. Just in time for C...
2,1,@mrdahl87 We are rumored to have talked to Erv...
3,1,Hey there! Nice to see you Minnesota/ND Winter...
4,0,3 episodes left I'm dying over here
5,1,I can't breathe! was chosen as the most notabl...
...,...,...
3830,0,@banditelli regarding what the PSU president does
3831,0,@banditelli But still bothers me that I see no...
3832,0,well now that i've listened to all of into the...


In [23]:
df_test.reset_index()

,index,label,tweet
0,1,0,@Callisto1947 Can U Help?||More conservatives ...
1,2,1,"Just walked in to #Starbucks and asked for a ""..."
2,3,0,#NOT GONNA WIN http://t.co/Mc9ebqjAqj
3,4,0,@mickymantell He is exactly that sort of perso...
4,5,1,So much #sarcasm at work mate 10/10 #boring 10...
...,...,...,...
779,780,0,"If you drag yesterday into today, your tomorro..."
780,781,0,Congrats to my fav @JennyBrew & her team & my ...
781,782,0,@allkpop: Jessica sheds tears at her fan signi...
782,783,1,#Irony: al jazeera is pro Anti - #GamerGate be...


In [15]:
# Train dev split
from sklearn.model_selection import train_test_split

train_df, dev_df = train_test_split(
    df, stratify=df["label"], random_state=20202021,
    test_size=0.20,
)

train_df.shape, dev_df.shape, df_test.shape

((3053, 2), (764, 2), (784, 2))

In [24]:
# Rename tweet to text

train_df = train_df.rename(columns={"tweet": "text"})
dev_df = dev_df.rename(columns={"tweet": "text"})
df_test = df_test.rename(columns={"tweet": "text"})


In [28]:
# 
from datasets import DatasetDict, Dataset, Features, Value, ClassLabel
from pysentimiento.irony import id2label

features = Features({
    'index': Value(dtype="int32"),
    'label': ClassLabel(num_classes=2, names=["not ironic", "ironic"]),
    'text': Value('string'),
})


Dataset.from_pandas(train_df.reset_index(), features=features),

ds = DatasetDict(
    train=Dataset.from_pandas(train_df, features=features),
    dev=Dataset.from_pandas(dev_df, features=features),
    test=Dataset.from_pandas(df_test, features=features),
)


In [29]:
ds["train"].features

{'index': Value(dtype='int32', id=None),
 'label': ClassLabel(names=['not ironic', 'ironic'], id=None),
 'text': Value(dtype='string', id=None)}

In [31]:
import pandas as pd
pd.options.display.max_colwidth = 150

train_df.sample(10)

,label,text
index,,
361,1,Let's all do only squats and deadlifts until we hurt ourselves --really really good trainers #fitnessmotivation
1536,0,@craigcalcaterra Let me put my bet in with whoever would take action on such a thing... I'd think Oliva & Howsam should be easy.
1503,0,I got to meet a real CSI tonight #cool but the little jerks who broke into his office at 2am cool:pouting_face:
2425,0,Better than yourself should be the score. Not better than other ones .
945,0,@GazTheGooner84 oh right do you know John Parker the pisshead @ParkzTheDon
1096,1,@DalailamaQuots I don't recall dalai lama talking about busty girls and celebs but apart from that....
2765,1,These Student Athletes Really Picked The Wrong Class To Cheat In http://t.co/JWeNktzy7M via @MTVNews
3214,0,"@georgenickss happy birthday young one, have a gooden lad!x"
1641,0,You know you love an author when you figure out how a book ends long before it does but can't wait to see how the author actually ends it


In [32]:
train_df["label"].value_counts()

0    1532
1    1521
Name: label, dtype: int64

It seems balanced.


In [34]:
df_test["label"].value_counts()



0    473
1    311
Name: label, dtype: int64

In [35]:
ds.push_to_hub("pysentimiento/en_irony", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]